# `Nuqleon.Linq.Expressions.Bonsai`

Provides a lightweight object model for expression trees with a lightweight representation of reflection.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net50/Nuqleon.Linq.Expressions.Bonsai.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Linq.Expressions.Bonsai,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

# `ExpressionSlim`

`System.Linq.Expressions.ExpressionSlim` is the base type for a lightweight representation of an expression tree, also referred to as a "Bonsai expression tree". Its lightweight nature stems from being decoupled from .NET reflection by using `Slim` variants of `Type` and the `MemberInfo` class hierarchy.

## Crafting a simple expression tree

To illustrate the manual creation of a Bonsai tree, let's consider the simple expression `1 + 2`. To represent this expression, we need a `BinaryExpression` of type `Add` and two `ConstantExpression`s holding values `1` and `2`. Let's start by crafting these constant nodes first.

Constant expressions consist of a value and a type. Rather than using `System.Object` for the value and `System.Type` for the type, Bonsai expressions abstract over the CLR type system and use alternative representations named `System.ObjectSlim` and `System.TypeSlim`. We'll start by creating a "simple type" representing `int`.

In [ ]:
using System.Reflection;

var asm = new AssemblySlim(typeof(int).Assembly.FullName);
var int32 = TypeSlim.Simple(asm, typeof(int).FullName);

Console.WriteLine(int32);

System.Int32


Other not-so-simple kinds of types include arrays, references, pointers, and generic types. All of these can be constructed using factory methods on `TypeSlim`. For a simple type, we simply have a name and an assembly.

> **Note:** While we use the notion of an *assembly*, Bonsai trees are decoupled from the CLR type system and can be rebound in other environments. Assemblies just play the role of holders for types and can be implemented in a variety of ways. For example, when binding a Bonsai tree in JavaScript, assemblies may get mapped onto modules.

Next, we can construct two `ObjectSlim` instances using the type constructed above, containing values of `1` and `2`. There are two overloads for the `Create` factory, which we'll explain a bit further on. For now, we'll use the overload that takes an `object` to hold the underlying CLR value, the `TypeSlim` to represent its type, and a `Type` to indicate the CLR type. This may look redundant, but we'll clarify this aspect later.

In [ ]:
var obj1 = ObjectSlim.Create(1, int32, typeof(int));
var obj2 = ObjectSlim.Create(2, int32, typeof(int));

Console.WriteLine(obj1);
Console.WriteLine(obj2);

1


2


Now that we have our values, we can start composing a tree on top. To do so, we can use familiar factory methods on `ExpressionSlim`.

In [ ]:
using System.Linq.Expressions;

var sum = ExpressionSlim.Add(ExpressionSlim.Constant(obj1), ExpressionSlim.Constant(obj2));

Console.WriteLine(sum);

1 + 2


Finally note that slim expression trees as well as slim types do have a `ToCSharpString` which provides a friendly string representation for debugging purposes.

In [ ]:
Console.WriteLine(sum.ToCSharpString());

1 + 2


## Crafting a lambda expression tree

To make things a little more interesting, we'll construct a Bonsai lambda expression tree representing `(int x, int y) => x + y`. To do so, we first need to craft a slightly more complex `TypeSlim` that's equivalent to `Func<int, int, int>`, which is a constructed generic type over the `Func<,,>` open generic type.

In [ ]:
var func3 = TypeSlim.GenericDefinition(new AssemblySlim(typeof(Func<,,>).Assembly.FullName), typeof(Func<,,>).FullName);
var func_int_int_int = TypeSlim.Generic(func3, int32, int32, int32);

Console.WriteLine(func_int_int_int.ToCSharpString());

System.Func<int, int, int>


Next, we need to construct two lambda parameters `x` and `y`.

In [ ]:
var px = ExpressionSlim.Parameter(int32, "x");
var py = ExpressionSlim.Parameter(int32, "y");

Console.WriteLine(px.ToCSharpString());
Console.WriteLine(py.ToCSharpString());

x


y


And finally, we can construct our lambda expression using `ExpressionSlim.Lambda`.

In [ ]:
var f = ExpressionSlim.Lambda(func_int_int_int, ExpressionSlim.Add(px, py), px, py);

Console.WriteLine(f.ToCSharpString());

(x, y) => x + y


## Conversion between `Expression` and `ExpressionSlim`

Obviously, hand-crafting these expression trees using factory methods can get quite bewildering. Luckily, the Bonsai expression tree library supports conversions between `Expression` and `ExpressionSlim`, as well as various auxiliary types such as `Type` and `TypeSlim`.

As a first example, we'll convert the `LambdaExpressionSlim` expression we created above to a `LambdaExpression`. This conversion involves binding the slim reflection members to the CLR type system. Under the hood, the string-based representations of types and assemblies are resolved through .NET reflection APIs.

In [ ]:
var g = (Expression<Func<int, int, int>>)f.ToExpression();

Console.WriteLine(g);

(x, y) => (x + y)


Note that the resulting expression is fully functional. For example, we can compile and evaluate it.

In [ ]:
var compiled = g.Compile();

Console.WriteLine(compiled(1, 2));

3


Under the hood, the `ToExpression` conversion is using an `ExpressionSlimToExpressionConverter` utility that uses an abstraction over reflection referred to as an "inverted type space". The following is an explicit instantiation of this utility, showing all the customization points which can be used to perform binding redirection, caching, etc.

In [ ]:
IExpressionFactory factory = ExpressionFactory.Instance;
IReflectionProvider provider = DefaultReflectionProvider.Instance;

var convertFromSlim = new ExpressionSlimToExpressionConverter(new InvertedTypeSpace(provider), factory);

var g = (Expression<Func<int, int, int>>)convertFromSlim.Visit(f);

Console.WriteLine(g);

(x, y) => (x + y)


In the opposite direction, we can take a LINQ expression tree of type `Expression` and convert it to `ExpressionSlim`. This is entirely symmetric using a `ToExpression` extension method.

In [ ]:
var h = (LambdaExpressionSlim)g.ToExpressionSlim();

Console.WriteLine(h.ToCSharpString());

(x, y) => x + y


Analogous to the opposite conversion, the `ToExpression` extension method is a convenience method that's based on an `ExpressionToExpressionSlimConverter`.

In [ ]:
IExpressionSlimFactory factory = ExpressionSlimFactory.Instance;

var convertToSlim = new ExpressionToExpressionSlimConverter(new TypeSpace(), factory);

var h = (LambdaExpressionSlim)convertToSlim.Visit(g);

Console.WriteLine(h.ToCSharpString());

(x, y) => x + y


## Other topics

This section contains a list of topics to be documented in more detail:

* `TypeUnifier` and wildcards.
* Support for structural types.
* `Lift` and `Reduce` on `ObjectSlim`.
* Utilities in `CompilerServices`.
* Details of `[Inverted]TypeSpace`.
* Equality comparers, visitors, etc.
* `IExpressionSerializer`.